# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [18]:
!pip install chromadb langchain langchain_community langchain_openai

In [31]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [20]:
# import os
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())

# OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [66]:
# If you're using colab, run this
import os
api_key = os.environ["OPENAI_API_KEY"]

In [67]:
llm = OpenAI(temperature=0, openai_api_key=api_key)

In [68]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [73]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(openai_api_key=api_key)
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [47]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [48]:
from langchain_community.document_loaders import WebBaseLoader

In [49]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [50]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [51]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [52]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [53]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [54]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.
Thought: I should use the Ruff QA System to get more information about Ketanji Brown Jackson.
Action: Ruff QA System
Action Input: "Who is Ketanji Brown Jackson?"
Observation:  I don't know.
Thought: I should use the State of Union QA System again to get more information about Ketanji Brown Jackson.
Action: State of Union QA System
Action Input: "Who is Ketanji Brown Jackson?"
Observation:  Ketanji Brown Jackson is a Circuit Court of Appeals Judge who was nominated by President Biden to serve on th

{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': 'Ketanji Brown Jackson is a Circuit Court of Appeals Judge who was nominated by President Biden to serve on the United States Supreme Court.'}

In [55]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8 before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff and Flake8 are both linters for Python code, but they have some key differences. Ruff uses different rule codes and prefixes than Flake8, which helps minimize conflicts across plugins and allows for easier toggling of individual plugins. Ruff also has a larger rule set, implementing over 800 rules compared to Flake8's ~409. However, Ruff does not support custom lint rules, while Flake8 does. Additionally, Ruff has some minor differences in its rule set, such as not including all of the 'opinionated' rules from flake8-bugbear. Ruff also has the capability to automatically fix its own lint violations, while Flake8 does not. Overall, Ruff can be used as a drop-in replacement for Flake8 in certain cases, but they are not identical tools and 

{'input': 'Why use ruff over flake8?',
 'output': "Ruff has a larger rule set, supports automatic fixing of lint violations, and has some minor differences in its rule set compared to Flake8. However, Flake8 supports custom lint rules and includes all of the 'opinionated' rules from flake8-bugbear. The decision between using Ruff or Flake8 ultimately depends on the specific needs and preferences of the user."}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [56]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [57]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [58]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  Biden mentioned that he nominated Ketanji Brown Jackson to serve on the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " Biden mentioned that he nominated Ketanji Brown Jackson to serve on the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [59]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8 before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff and Flake8 are both linters for Python code, but they have some key differences. Ruff uses different rule codes and prefixes than Flake8, which helps minimize conflicts across plugins and allows for easier toggling of individual plugins. Ruff also has a larger rule set, implementing over 800 rules compared to Flake8's ~409. However, Ruff does not support custom lint rules, while Flake8 does. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 relies on plugins for this functionality. Finally, Ruff is designed to be used alongside a type checker, while Flake8 does not have this capability.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': " Ruff and Flake8 are both linters for Python code, but they have some key differences. Ruff uses different rule codes and prefixes than Flake8, which helps minimize conflicts across plugins and allows for easier toggling of individual plugins. Ruff also has a larger rule set, implementing over 800 rules compared to Flake8's ~409. However, Ruff does not support custom lint rules, while Flake8 does. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 relies on plugins for this functionality. Finally, Ruff is designed to be used alongside a type checker, while Flake8 does not have this capability."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [60]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [61]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [63]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer this question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff uses the colored crate to automatically detect whether the output stream supports color.
Thought: I should use the State of Union QA System to check if the president mentioned this tool.
Action: State of Union QA System
Action Input: Did the president mention the colored crate in the state of the union?
Observation:  No, the president did not mention a colored crate in the State of the Union address.
Thought: I now know the final answer.
Final Answer: The final answer is that the president did not mention the colored crate in the State of the Union address.

> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'The final answer is that the president did not mention the colored crate in the State of the Union address.'}